In [13]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os

import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
import re
import itertools

In [2]:
def spectral_entropy(x, sf=100, method='fft', nperseg=None, normalize=False,axis=-1):
    """Spectral Entropy.
    Parameters
    ----------
    x : list or np.array
        1D or N-D data.
    sf : float
        Sampling frequency, in Hz.
    method : str
        Spectral estimation method:
        * ``'fft'`` : Fourier Transform (:py:func:`scipy.signal.periodogram`)
        * ``'welch'`` : Welch periodogram (:py:func:`scipy.signal.welch`)
    nperseg : int or None
        Length of each FFT segment for Welch method.
        If None (default), uses scipy default of 256 samples.
    normalize : bool
        If True, divide by log2(psd.size) to normalize the spectral entropy
        between 0 and 1. Otherwise, return the spectral entropy in bit.
    axis : int
        The axis along which the entropy is calculated. Default is -1 (last).
    Returns
    -------
    se : float
        Spectral Entropy
    Notes
    -----
    Spectral Entropy is defined to be the Shannon entropy of the power
    spectral density (PSD) of the data:
    .. math:: H(x, sf) =  -\\sum_{f=0}^{f_s/2} P(f) \\log_2[P(f)]
    Where :math:`P` is the normalised PSD, and :math:`f_s` is the sampling
    frequency.
    References
    ----------
    - Inouye, T. et al. (1991). Quantification of EEG irregularity by
      use of the entropy of the power spectrum. Electroencephalography
      and clinical neurophysiology, 79(3), 204-210.
    - https://en.wikipedia.org/wiki/Spectral_density
    - https://en.wikipedia.org/wiki/Welch%27s_method
    Examples
    --------
    Spectral entropy of a pure sine using FFT
    >>> import numpy as np
    >>> import entropy as ent
    >>> sf, f, dur = 100, 1, 4
    >>> N = sf * dur # Total number of discrete samples
    >>> t = np.arange(N) / sf # Time vector
    >>> x = np.sin(2 * np.pi * f * t)
    >>> np.round(ent.spectral_entropy(x, sf, method='fft'), 2)
    0.0
    Spectral entropy of a random signal using Welch's method
    >>> np.random.seed(42)
    >>> x = np.random.rand(3000)
    >>> ent.spectral_entropy(x, sf=100, method='welch')
    6.980045662371389
    Normalized spectral entropy
    >>> ent.spectral_entropy(x, sf=100, method='welch', normalize=True)
    0.9955526198316071
    Normalized spectral entropy of 2D data
    >>> np.random.seed(42)
    >>> x = np.random.normal(size=(4, 3000))
    >>> np.round(ent.spectral_entropy(x, sf=100, normalize=True), 4)
    array([0.9464, 0.9428, 0.9431, 0.9417])
    Fractional Gaussian noise with H = 0.5
    >>> import stochastic.processes.noise as sn
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.5, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9505
    Fractional Gaussian noise with H = 0.9
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.9, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.8477
    Fractional Gaussian noise with H = 0.1
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.1, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9248
    """
    x = np.asarray(x)
    #sf = np.array(sf, dtype=np.float64)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf, axis=axis)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
    if normalize:
        se /= np.log2(psd_norm.shape[axis])
    return se

In [102]:
def statf(c, dfa, dfg, leng):
    
    # acc
    mean_a = dfa.mean()
    var_a = dfa.var()
    std_a = dfa.std()
    mx_a = dfa.max()
    mn_a = dfa.min()
    rng_a = mx_a-mn_a
    #mode = df[col].mode()
    #n_z = ((df[col][:-1] * df[col][1:]) < 0).sum()
    
    #gyro
    mean_g = dfg.mean()
    var_g = dfg.var()
    std_g = dfg.std()
    mx_g = dfg.max()
    mn_g = dfg.min()
    rng_g = mx_g-mn_g
    
    f_a=abs(np.fft.fft(dfa))
    f_g=abs(np.fft.fft(dfg))
    num=dfa.shape[0]
    num = len(dfa)
    
    #fs=1./diff(t)
    
    freq = [i / num for i in list(range(num))]
    
    spectrum_a = f_a.real*f_a.real+f_a.imag*f_a.imag
    dc_comp_a = spectrum_a[0]
    spectrum_a = np.multiply(spectrum_a,spectrum_a)  # get the sectrum square
    spec_energy_a = spectrum_a.sum()/num
    
    spectrum_g = f_g.real*f_g.real+f_g.imag*f_g.imag
    dc_comp_g = spectrum_g[0]
    spectrum_g = np.multiply(spectrum_g,spectrum_g)  # get the sectrum square
    spec_energy_g = spectrum_g.sum()/num
    
    # spectral entropy
    
    
    ts = []
    t = 0
    T = (1/50)
    for l in range(leng):
        ts.append(t)
        t = t + T
    
    #ts_shift = ts
    #ts_shift.insert(0,ts_shift.pop())
    
    #print(ts)
    #print(ts_shift)
    
#     freq1 = np.subtract(ts,ts_shift)
#     freq1 = freq1.dropna(axis = 0, how = 'all')
#     print(freq1)
#     mean_freq = np.mean(freq1)
#     print(mean_freq)
    mean_freq = 50
    spec_entropy_a = spectral_entropy(dfa, mean_freq, method='fft', nperseg=None, normalize=False,axis=-1)
    spec_entropy_g = spectral_entropy(dfg, mean_freq, method='fft', nperseg=None, normalize=False,axis=-1)
    
    # poer spectral density using LombScargle Periodogram
#     print(len(ts))
#     print(len(dfa))
    
    frequency_a, power_a = LombScargle(ts, dfa).autopower()

    max_psd_a = np.max(power_a)
    min_psd_a = np.min(power_a)
    min_max_psd_a = min_psd_a/max_psd_a
    
    max_xas_a = np.sqrt(max_psd_a)
    min_xas_a = np.sqrt(min_psd_a)
    min_max_xas_a = min_xas_a/max_xas_a
    
    frequency_g, power_g = LombScargle(ts, dfg).autopower()

    max_psd_g = np.max(power_g)
    min_psd_g = np.min(power_g)
    min_max_psd_g = min_psd_g/max_psd_g
    
    max_xas_g = np.sqrt(max_psd_g)
    min_xas_g = np.sqrt(min_psd_g)
    min_max_xas_g = min_xas_g/max_xas_g

    return np.array([mean_a ,var_a ,std_a ,mx_a ,mn_a ,rng_a ,dc_comp_a ,spec_energy_a ,spec_entropy_a , 
                     max_psd_a ,min_psd_a ,min_max_psd_a ,max_xas_a ,min_xas_a ,min_max_xas_a ,
                    mean_g ,var_g ,std_g ,mx_g ,mn_g ,rng_g ,dc_comp_g ,spec_energy_g ,spec_entropy_g , 
                     max_psd_g ,min_psd_g ,min_max_psd_g ,max_xas_g ,min_xas_g ,min_max_xas_g ])
    

In [103]:
path = os.path.abspath(os.getcwd())+'\\inertial'
files = os.listdir(path)
i = 0

for file in files:
    #print(file)
    df = sio.loadmat(path+'//'+file)
    #print(df.keys())
    df = df['d_iner']
    #print(data.shape)
    #print(data[0])
    i+=1
    if(i==10):
        break
    
    # data[:][]
    
    

df = df.transpose()
gh = statf('x',df[0],df[3],len(df[0]))
#print(df[0])
df.shape

<ipython-input-102-7dc1a520ee8a>:76: RuntimeWarning: invalid value encountered in sqrt
  min_xas_a = np.sqrt(min_psd_a)


(6, 201)

In [65]:
f = 0
t = 1/50


In [104]:
data = np.array([])
#sensor = np.array([])
k = 0

# for each sensor position
    
sensor = np.array([])
k = 0

path = os.path.abspath(os.getcwd())+'\\inertial'
files = os.listdir(path)

# for each time seq
for file in files:

    # namemaking
    print(file)
    x = re.split("_|\.",file)
    act = x[0]
    name = x[1]
    time = x[2]
    mat_filename = file
    filename = act+"_"+name+"_"+time
    
    action = act[1:] 

    df = sio.loadmat(path+'//'+file)
    df = df['d_iner']
    
    df = df.transpose()
    
    leng = len(df[0])
#     print(leng)
#     print()
    
    x = statf('x',df[0],df[3],leng)
    y = statf('y',df[1],df[4],leng)
    z = statf('z',df[2],df[5],leng)
    
    df_a_m =  np.sqrt(df[0]*df[0] + df[1]*df[1] + df[2]*df[2])
    df_g_m =  np.sqrt(df[3]*df[3] + df[4]*df[4] + df[5]*df[5])
    
    m = statf('m', df_a_m, df_g_m, leng)
    #act = j+1
    print(filename)

    l = [action,mat_filename,filename]
    sen = np.append(np.concatenate((x, y, z, m), axis = 0),l)                  
    # add all the features together to make a row

    #dat = np.append(np.concatenate((x, y, z, m), axis = 0),act)
    #dat = np.append(dat,i)

    # add all the new rows
    if(k==0):
        sensor = sen
    else: 
        sensor =  np.vstack([sensor,sen])
    k+=1
    #print(k)

print(sensor.shape)

a10_s1_t1_inertial.mat
a10_s1_t1
a10_s1_t2_inertial.mat
a10_s1_t2
a10_s1_t3_inertial.mat
a10_s1_t3
a10_s1_t4_inertial.mat
a10_s1_t4
a10_s2_t1_inertial.mat
a10_s2_t1
a10_s2_t2_inertial.mat
a10_s2_t2
a10_s2_t3_inertial.mat
a10_s2_t3
a10_s2_t4_inertial.mat
a10_s2_t4
a10_s3_t1_inertial.mat
a10_s3_t1
a10_s3_t2_inertial.mat
a10_s3_t2
a10_s3_t3_inertial.mat
a10_s3_t3
a10_s3_t4_inertial.mat


<ipython-input-102-7dc1a520ee8a>:76: RuntimeWarning: invalid value encountered in sqrt
  min_xas_a = np.sqrt(min_psd_a)
<ipython-input-102-7dc1a520ee8a>:86: RuntimeWarning: invalid value encountered in sqrt
  min_xas_g = np.sqrt(min_psd_g)
<ipython-input-2-b64774160073>:88: RuntimeWarning: divide by zero encountered in log2
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
<ipython-input-2-b64774160073>:88: RuntimeWarning: invalid value encountered in multiply
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)


a10_s3_t4
a10_s4_t1_inertial.mat
a10_s4_t1
a10_s4_t2_inertial.mat
a10_s4_t2
a10_s4_t3_inertial.mat
a10_s4_t3
a10_s4_t4_inertial.mat
a10_s4_t4
a10_s5_t1_inertial.mat
a10_s5_t1
a10_s5_t2_inertial.mat
a10_s5_t2
a10_s5_t3_inertial.mat
a10_s5_t3
a10_s5_t4_inertial.mat
a10_s5_t4
a10_s6_t1_inertial.mat
a10_s6_t1
a10_s6_t2_inertial.mat
a10_s6_t2
a10_s6_t3_inertial.mat
a10_s6_t3
a10_s6_t4_inertial.mat
a10_s6_t4
a10_s7_t1_inertial.mat
a10_s7_t1
a10_s7_t2_inertial.mat
a10_s7_t2
a10_s7_t3_inertial.mat
a10_s7_t3
a10_s7_t4_inertial.mat
a10_s7_t4
a10_s8_t1_inertial.mat
a10_s8_t1
a10_s8_t2_inertial.mat
a10_s8_t2
a10_s8_t3_inertial.mat
a10_s8_t3
a10_s8_t4_inertial.mat
a10_s8_t4
a11_s1_t1_inertial.mat
a11_s1_t1
a11_s1_t2_inertial.mat
a11_s1_t2
a11_s1_t3_inertial.mat
a11_s1_t3
a11_s1_t4_inertial.mat
a11_s1_t4
a11_s2_t1_inertial.mat
a11_s2_t1
a11_s2_t2_inertial.mat
a11_s2_t2
a11_s2_t3_inertial.mat
a11_s2_t3
a11_s2_t4_inertial.mat
a11_s2_t4
a11_s3_t1_inertial.mat
a11_s3_t1
a11_s3_t2_inertial.mat
a11_s3_t2


a18_s3_t4_inertial.mat
a18_s3_t4
a18_s4_t1_inertial.mat
a18_s4_t1
a18_s4_t2_inertial.mat
a18_s4_t2
a18_s4_t3_inertial.mat
a18_s4_t3
a18_s4_t4_inertial.mat
a18_s4_t4
a18_s5_t1_inertial.mat
a18_s5_t1
a18_s5_t2_inertial.mat
a18_s5_t2
a18_s5_t3_inertial.mat
a18_s5_t3
a18_s5_t4_inertial.mat
a18_s5_t4
a18_s6_t1_inertial.mat
a18_s6_t1
a18_s6_t2_inertial.mat
a18_s6_t2
a18_s6_t3_inertial.mat
a18_s6_t3
a18_s6_t4_inertial.mat
a18_s6_t4
a18_s7_t1_inertial.mat
a18_s7_t1
a18_s7_t2_inertial.mat
a18_s7_t2
a18_s7_t3_inertial.mat
a18_s7_t3
a18_s7_t4_inertial.mat
a18_s7_t4
a18_s8_t1_inertial.mat
a18_s8_t1
a18_s8_t2_inertial.mat
a18_s8_t2
a18_s8_t3_inertial.mat
a18_s8_t3
a18_s8_t4_inertial.mat
a18_s8_t4
a19_s1_t1_inertial.mat
a19_s1_t1
a19_s1_t2_inertial.mat
a19_s1_t2
a19_s1_t3_inertial.mat
a19_s1_t3
a19_s1_t4_inertial.mat
a19_s1_t4
a19_s2_t1_inertial.mat
a19_s2_t1
a19_s2_t2_inertial.mat
a19_s2_t2
a19_s2_t3_inertial.mat
a19_s2_t3
a19_s2_t4_inertial.mat
a19_s2_t4
a19_s3_t1_inertial.mat
a19_s3_t1
a19_s3_t2_

a25_s2_t4
a25_s3_t1_inertial.mat
a25_s3_t1
a25_s3_t2_inertial.mat
a25_s3_t2
a25_s3_t3_inertial.mat
a25_s3_t3
a25_s3_t4_inertial.mat
a25_s3_t4
a25_s4_t1_inertial.mat
a25_s4_t1
a25_s4_t2_inertial.mat
a25_s4_t2
a25_s4_t3_inertial.mat
a25_s4_t3
a25_s4_t4_inertial.mat
a25_s4_t4
a25_s5_t1_inertial.mat
a25_s5_t1
a25_s5_t2_inertial.mat
a25_s5_t2
a25_s5_t3_inertial.mat
a25_s5_t3
a25_s5_t4_inertial.mat
a25_s5_t4
a25_s6_t1_inertial.mat
a25_s6_t1
a25_s6_t2_inertial.mat
a25_s6_t2
a25_s6_t3_inertial.mat
a25_s6_t3
a25_s6_t4_inertial.mat
a25_s6_t4
a25_s7_t1_inertial.mat
a25_s7_t1
a25_s7_t2_inertial.mat
a25_s7_t2
a25_s7_t3_inertial.mat
a25_s7_t3
a25_s7_t4_inertial.mat
a25_s7_t4
a25_s8_t1_inertial.mat
a25_s8_t1
a25_s8_t2_inertial.mat
a25_s8_t2
a25_s8_t3_inertial.mat
a25_s8_t3
a25_s8_t4_inertial.mat
a25_s8_t4
a26_s1_t1_inertial.mat
a26_s1_t1
a26_s1_t2_inertial.mat
a26_s1_t2
a26_s1_t3_inertial.mat
a26_s1_t3
a26_s1_t4_inertial.mat
a26_s1_t4
a26_s2_t1_inertial.mat
a26_s2_t1
a26_s2_t2_inertial.mat
a26_s2_t2


a7_s3_t4
a7_s4_t1_inertial.mat
a7_s4_t1
a7_s4_t2_inertial.mat
a7_s4_t2
a7_s4_t3_inertial.mat
a7_s4_t3
a7_s4_t4_inertial.mat
a7_s4_t4
a7_s5_t1_inertial.mat
a7_s5_t1
a7_s5_t2_inertial.mat
a7_s5_t2
a7_s5_t3_inertial.mat
a7_s5_t3
a7_s5_t4_inertial.mat
a7_s5_t4
a7_s6_t1_inertial.mat
a7_s6_t1
a7_s6_t2_inertial.mat
a7_s6_t2
a7_s6_t3_inertial.mat
a7_s6_t3
a7_s6_t4_inertial.mat
a7_s6_t4
a7_s7_t1_inertial.mat
a7_s7_t1
a7_s7_t2_inertial.mat
a7_s7_t2
a7_s7_t3_inertial.mat
a7_s7_t3
a7_s7_t4_inertial.mat
a7_s7_t4
a7_s8_t1_inertial.mat
a7_s8_t1
a7_s8_t2_inertial.mat
a7_s8_t2
a7_s8_t3_inertial.mat
a7_s8_t3
a7_s8_t4_inertial.mat
a7_s8_t4
a8_s1_t1_inertial.mat
a8_s1_t1
a8_s1_t2_inertial.mat
a8_s1_t2
a8_s1_t3_inertial.mat
a8_s1_t3
a8_s2_t1_inertial.mat
a8_s2_t1
a8_s2_t2_inertial.mat
a8_s2_t2
a8_s2_t3_inertial.mat
a8_s2_t3
a8_s2_t4_inertial.mat
a8_s2_t4
a8_s3_t1_inertial.mat
a8_s3_t1
a8_s3_t2_inertial.mat
a8_s3_t2
a8_s3_t3_inertial.mat
a8_s3_t3
a8_s3_t4_inertial.mat
a8_s3_t4
a8_s4_t1_inertial.mat
a8_s4_t1

In [105]:

# mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas

col = ['mean','var','std','max','min','range','dc_comp','spec_energy','spec_entropy','max_psd','min_psd','min_max_psd','max_xas','min_xas','min_max_xas']
axisa = ['_a_x','_g_x','_a_y','_g_y','_a_z','_g_z','_a_m','_g_m']
col1 = ['activity','mat_filename','filename']

col_name = []


for x in axisa:
    for (c) in (col):
        col_name.append(c+x)
        
for c1 in col1:
    col_name.append(c1)

    
# col_name.append('activity')
# col_name.append('position')
print(col_name)

['mean_a_x', 'var_a_x', 'std_a_x', 'max_a_x', 'min_a_x', 'range_a_x', 'dc_comp_a_x', 'spec_energy_a_x', 'spec_entropy_a_x', 'max_psd_a_x', 'min_psd_a_x', 'min_max_psd_a_x', 'max_xas_a_x', 'min_xas_a_x', 'min_max_xas_a_x', 'mean_g_x', 'var_g_x', 'std_g_x', 'max_g_x', 'min_g_x', 'range_g_x', 'dc_comp_g_x', 'spec_energy_g_x', 'spec_entropy_g_x', 'max_psd_g_x', 'min_psd_g_x', 'min_max_psd_g_x', 'max_xas_g_x', 'min_xas_g_x', 'min_max_xas_g_x', 'mean_a_y', 'var_a_y', 'std_a_y', 'max_a_y', 'min_a_y', 'range_a_y', 'dc_comp_a_y', 'spec_energy_a_y', 'spec_entropy_a_y', 'max_psd_a_y', 'min_psd_a_y', 'min_max_psd_a_y', 'max_xas_a_y', 'min_xas_a_y', 'min_max_xas_a_y', 'mean_g_y', 'var_g_y', 'std_g_y', 'max_g_y', 'min_g_y', 'range_g_y', 'dc_comp_g_y', 'spec_energy_g_y', 'spec_entropy_g_y', 'max_psd_g_y', 'min_psd_g_y', 'min_max_psd_g_y', 'max_xas_g_y', 'min_xas_g_y', 'min_max_xas_g_y', 'mean_a_z', 'var_a_z', 'std_a_z', 'max_a_z', 'min_a_z', 'range_a_z', 'dc_comp_a_z', 'spec_energy_a_z', 'spec_entrop

In [106]:
len(col_name)

123

In [107]:
xD = pd.DataFrame(sensor)
xD.columns = col_name
xD.to_csv('utd_mhad_sensor.csv', index=False)

In [108]:
xD.head()

,mean_a_x,var_a_x,std_a_x,max_a_x,min_a_x,range_a_x,dc_comp_a_x,spec_energy_a_x,spec_entropy_a_x,max_psd_a_x,...,spec_entropy_g_m,max_psd_g_m,min_psd_g_m,min_max_psd_g_m,max_xas_g_m,min_xas_g_m,min_max_xas_g_m,activity,mat_filename,filename
0,-0.6124662239583334,0.24983328178969463,0.49983325398546125,0.38916,-1.701904,2.0910640000000003,13828.234770055227,1044899.0661812028,2.518966683270971,0.5225582493534621,...,2.3225018890634144,0.782689026794682,-0.07917754718201009,-0.1011609265895333,0.8846971384573831,nan,nan,10,a10_s1_t1_inertial.mat,a10_s1_t1
1,-0.6033667914438504,0.40638871331597254,0.6374862455896382,0.510742,-2.227051,2.737793,12730.516379568098,1027958.8903217375,2.127660321666135,0.5167270282286409,...,2.259249825365963,0.680180252774153,8.568289115472681e-08,1.2597085964384348e-07,0.8247304121797334,0.00029271640055645464,0.0003549237377858002,10,a10_s1_t2_inertial.mat,a10_s1_t2
2,-0.5203425333333332,0.45533390237526433,0.6747843376777979,0.616211,-2.222168,2.8383789999999998,10295.510284638434,769229.4895514789,2.1393277818327476,0.44824162855125577,...,2.225795580140987,0.6308245987035574,6.450605064186578e-08,1.022567141079085e-07,0.7942446718131368,0.0002539804138941934,0.00031977603741979873,10,a10_s1_t3_inertial.mat,a10_s1_t3
3,-0.5992758579234972,0.37053163153707275,0.6087130945996421,0.539551,-2.066895,2.606446,12026.956608220326,916036.698124122,2.090633031234528,0.47215182875468314,...,2.338178470770278,0.7238136559947296,4.627308394706906e-08,6.392955364109073e-08,0.8507723878892225,0.0002151117940678034,0.00025284294263651244,10,a10_s1_t4_inertial.mat,a10_s1_t4
4,-0.4370316279069767,0.39235735308114067,0.6263843493264664,0.494873,-1.473389,1.9682620000000002,5650.444709913601,347925.9769859022,1.8325721562963546,0.7967273278198299,...,2.499988061153172,0.6928506358004736,-0.1331804203351036,-0.19222096863811894,0.832376498827588,nan,nan,10,a10_s2_t1_inertial.mat,a10_s2_t1


In [109]:
xD = xD.fillna(xD.mean())
is_NaN = xD.isnull()
row_has_NaN = is_NaN.any(axis=1)
xD[row_has_NaN]

,mean_a_x,var_a_x,std_a_x,max_a_x,min_a_x,range_a_x,dc_comp_a_x,spec_energy_a_x,spec_entropy_a_x,max_psd_a_x,...,spec_entropy_g_m,max_psd_g_m,min_psd_g_m,min_max_psd_g_m,max_xas_g_m,min_xas_g_m,min_max_xas_g_m,activity,mat_filename,filename


In [110]:
xD.columns = col_name
xD.to_csv('utd_mhad_sensor_nan.csv', index=False)